# Material de Referencia: Análisis del Compañero

**Equipo Original:** Franco Aguilar, Iván Mansilla y Ayrton Morrison  
**Propósito de este documento:** Recopilación de implementaciones útiles para complementar `Entregable.ipynb`

---

## Índice de contenidos por sección

1. **Sección 2.2 - Intervalos de Confianza**
   - Función `intervalo_confianza_95()` reutilizable
   - Aplicación a variables clave (IMC, FPG, HbA1c)
   - Interpretaciones clínicas con rangos de referencia

2. **Sección 2.3 - Test de Hipótesis**
   - Comparación PA sistólica (Welch t-test + Mann-Whitney)
   - ANOVA/Kruskal-Wallis (dieta vs actividad física)
   - Chi-cuadrado (antecedentes familiares vs GDM)

3. **Sección 2.4 - Evaluación de Normalidad**
   - Formato de presentación simplificado para Shapiro-Wilk

4. **Otras utilidades**
   - Análisis de outliers con IQR
   - Visualizaciones básicas compactas

---

**Nota:** Este notebook NO se ejecuta directamente. Es una referencia de código para integrar en `Entregable.ipynb`.

---

## 1. Sección 2.2 - Intervalos de Confianza

### 1.1 Función para calcular IC 95% de la media

**Ventajas:**
- Compacta y reutilizable
- Usa distribución t de Student correctamente
- Devuelve DataFrame legible
- Maneja NaN automáticamente

**Aplicación sugerida:**
- Usar para calcular IC de todas las variables continuas en `df_filter`
- Complementar con tu función `calculate_ic_std()` para IC de varianza
- Crear tabla resumen `ic_table_completa` para el informe

In [ ]:
from scipy import stats
import math
import pandas as pd
import numpy as np

def intervalo_confianza_95(x):
    """Calcula IC 95% para la media usando distribución t de Student
    
    Args:
        x (pd.Series): Serie de datos numéricos
        
    Returns:
        pd.Series: media, IC95_inf, IC95_sup
    """
    x = x.dropna().astype(float)
    n = len(x)
    
    if n < 2:
        return pd.Series({
            "media": np.nan,
            "IC95_inf": np.nan,
            "IC95_sup": np.nan
        })
    
    media = x.mean()
    desv = x.std(ddof=1)
    t_crit = stats.t.ppf(0.975, df=n-1)
    margen = t_crit * desv / math.sqrt(n)
    
    return pd.Series({
        "media": round(media, 3),
        "IC95_inf": round(media - margen, 3),
        "IC95_sup": round(media + margen, 3)
    })

# Ejemplo de uso:
# vars_interes = ["bmi_prepreg_kg_m2", "fpg_mmol_l", "hba1c_percent"]
# ic_table = df[vars_interes].apply(intervalo_confianza_95).T
# display(ic_table)

### 1.2 Interpretación Clínica de Intervalos de Confianza

**Modelo de interpretación del compañero:**

Este formato es excelente para la sección de conclusiones. Estructura clara:
1. Variable + IC numérico
2. Comparación con rangos clínicos de referencia
3. Conclusión sobre la población de estudio

---

#### Ejemplo 1: IMC Pregestacional

**IC 95%:** [26.031, 26.538] kg/m²

**Interpretación:**  
El intervalo de confianza está completamente en el rango de **sobrepeso** según clasificación OMS (25-29.9 kg/m²).  

**Conclusión poblacional:**  
La población de estudio, en promedio, presenta sobrepeso antes del embarazo. Este es un **factor de riesgo conocido** para el desarrollo de diabetes gestacional.

---

#### Ejemplo 2: Glucosa Plasmática en Ayunas (FPG)

**IC 95%:** [4.75, 4.85] mmol/L

**Interpretación:**  
El intervalo está por debajo del umbral diagnóstico de:
- Pre-diabetes: 5.6-6.9 mmol/L
- Diabetes: ≥7.0 mmol/L

**Conclusión poblacional:**  
En promedio, la población presenta **niveles normoglucémicos** durante el primer trimestre del embarazo.

---

#### Ejemplo 3: Hemoglobina Glicosilada (HbA1c)

**IC 95%:** [5.199, 5.239] %

**Interpretación:**  
Considerando los umbrales de:
- Pre-diabetes: >5.7%
- Diabetes: ≥6.5%

**Conclusión poblacional:**  
El intervalo indica que, en promedio, la población se encuentra **metabólicamente sana** en cuanto a control glucémico a largo plazo.

---

**📝 Acción recomendada:**  
Usar este formato para interpretar IC de **todas** las variables continuas en tu sección de conclusiones (Sección 4.2).

---

## 2. Sección 2.3 - Test de Hipótesis

### 2.1 Comparación de Presión Arterial Sistólica (GDM vs No-GDM)

**Características importantes:**
- Ejecuta AMBAS pruebas: Welch t-test (paramétrico) + Mann-Whitney U (no paramétrico)
- Proporciona robustez al análisis
- Formato claro de presentación de resultados

**Aplicación sugerida:**
- Integrar en tu celda 52 de `Entregable.ipynb`
- Extender a otras variables: `bmi_prepreg_kg_m2`, `fpg_mmol_l`, `hba1c_percent`, etc.
- Combinar con tu función `compare_two_groups_numeric()` existente

In [ ]:
import numpy as np
from scipy import stats

# Ejemplo: Comparación de presión arterial sistólica

# Filtrar grupos
gdm = df_filter.loc[df_filter["label_gdm"] == 1, "systolic_bp_mmHg"].dropna()
nogdm = df_filter.loc[df_filter["label_gdm"] == 0, "systolic_bp_mmHg"].dropna()

print("1) Comparación de presión arterial sistólica\n")
print(f"N GDM = {len(gdm)}, N No GDM = {len(nogdm)}")

# Welch t-test (no asume varianzas iguales)
t_stat, t_p = stats.ttest_ind(gdm, nogdm, equal_var=False, nan_policy="omit")

# Mann-Whitney U (alternativa no paramétrica)
u_stat, u_p = stats.mannwhitneyu(gdm, nogdm, alternative="two-sided")

print(f"Welch t-test: t = {t_stat:.3f}, p = {t_p:.4f}")
print(f"Mann–Whitney U: U = {u_stat:.3f}, p = {u_p:.4f}")

if t_p < 0.05:
    print("✅ Existe diferencia estadísticamente significativa en la PA sistólica entre GDM y no GDM.\n")
else:
    print("❌ No se observa diferencia estadísticamente significativa en la PA sistólica entre GDM y no GDM.\n")

### 2.2 ANOVA y Kruskal-Wallis: Dieta vs Actividad Física

**Características importantes:**
- Comparación de múltiples grupos (>2)
- Categorización automática de variable continua en terciles
- Ejecuta ANOVA (paramétrico) y Kruskal-Wallis (no paramétrico)

**📌 CRÍTICO:** Esta prueba está EXPLÍCITAMENTE pedida en las instrucciones:
> "Evaluar diferencias en dieta por nivel de actividad física (ANOVA / Kruskal–Wallis)"

**Aplicación directa:**
- Agregar como nueva celda en sección 2.3 de `Entregable.ipynb`
- Esta prueba completa uno de los requisitos faltantes más críticos

In [ ]:
# 2) Puntaje de dieta vs nivel de actividad física
print("2) Puntaje de dieta vs nivel de actividad física\n")

# Crear terciles de actividad física
q1, q2 = df_filter["physical_activity_level"].quantile([1/3, 2/3])

def clasificar_pal(x):
    """Clasifica nivel de actividad física en terciles"""
    if pd.isna(x): 
        return np.nan
    if x <= q1: 
        return "Baja"
    elif x <= q2: 
        return "Media"
    else: 
        return "Alta"

df_filter["PAL_categoria"] = df_filter["physical_activity_level"].apply(clasificar_pal)

# Preparar datos para análisis
tmp = df_filter[["diet_score_0_100", "PAL_categoria"]].dropna()
grupos = [g["diet_score_0_100"].values for _, g in tmp.groupby("PAL_categoria")]
labels = tmp["PAL_categoria"].unique()

print("Grupos:", labels)
print(f"N por grupo: {[len(g) for g in grupos]}")

# ANOVA (paramétrico)
f_stat, f_p = stats.f_oneway(*grupos)

# Kruskal-Wallis (no paramétrico)
kw_stat, kw_p = stats.kruskal(*grupos)

print(f"\nANOVA: F = {f_stat:.3f}, p = {f_p:.4f}")
print(f"Kruskal–Wallis: H = {kw_stat:.3f}, p = {kw_p:.4f}")

if f_p < 0.05:
    print("✅ Existen diferencias significativas en el puntaje de dieta según el nivel de actividad física.\n")
else:
    print("❌ No se encontraron diferencias significativas en el puntaje de dieta entre los niveles de actividad física.\n")

### 2.3 Prueba Chi-cuadrado: Antecedentes Familiares de DM2 vs GDM

**Características importantes:**
- Prueba de asociación para variables categóricas
- Usa tabla de contingencia (crosstab)
- Devuelve chi², p-value, grados de libertad y frecuencias esperadas

**📌 CRÍTICO:** Esta prueba está EXPLÍCITAMENTE pedida en las instrucciones:
> "Comparación de proporciones (antecedentes familiares vs GDM)"

**Aplicación sugerida:**
- Agregar como nueva celda en sección 2.3
- Extender a otras variables categóricas: `pcos`, `previous_gdm`, `smoking_first_trimester`
- Calcular Odds Ratio (OR) para cada comparación

In [ ]:
# 3) Antecedentes familiares de T2D vs GDM
print("3) Asociación entre antecedentes familiares de T2D y GDM\n")

# Crear tabla de contingencia
tabla = pd.crosstab(df_filter["family_history_t2d"], df_filter["label_gdm"])
print("Tabla de contingencia:")
print(tabla)

# Prueba chi-cuadrado
chi2, chi_p, dof, exp = stats.chi2_contingency(tabla)

print(f"\nChi-cuadrado = {chi2:.3f}")
print(f"Grados de libertad = {dof}")
print(f"p-value = {chi_p:.4f}")

# Calcular Odds Ratio (si tabla 2x2)
if tabla.shape == (2, 2):
    a, b = tabla.iloc[0, 0], tabla.iloc[0, 1]
    c, d = tabla.iloc[1, 0], tabla.iloc[1, 1]
    
    if b * c > 0:
        OR = (a * d) / (b * c)
        print(f"Odds Ratio = {OR:.3f}")
        
        if OR > 1:
            print(f"Los antecedentes familiares aumentan el riesgo en {OR:.2f}x")
        else:
            print(f"Los antecedentes familiares reducen el riesgo en {1/OR:.2f}x")

if chi_p < 0.05:
    print("\n✅ Existe asociación estadísticamente significativa entre tener antecedentes familiares de T2D y presentar GDM.")
else:
    print("\n❌ No se observa asociación significativa entre antecedentes familiares de T2D y GDM.")

### 2.4 Código Integrado: Todas las pruebas de hipótesis categóricas

**Aplicación sugerida:**
- Crear una celda nueva en sección 2.3 después de tu celda 52
- Este código ejecuta chi-cuadrado para TODAS las variables categóricas relevantes
- Genera tabla resumen con resultados consolidados

In [ ]:
# ===== ANÁLISIS COMPLETO: CHI-CUADRADO PARA VARIABLES CATEGÓRICAS =====
print("="*70)
print("PRUEBAS DE ASOCIACIÓN CHI-CUADRADO (Variables Categóricas vs GDM)")
print("="*70)

categorical_tests = [
    ("family_history_t2d", "Antecedentes familiares de DM2"),
    ("pcos", "Síndrome de ovario poliquístico"),
    ("previous_gdm", "Diabetes gestacional previa"),
    ("smoking_first_trimester", "Tabaquismo en 1er trimestre")
]

chi_results = []

for col, descripcion in categorical_tests:
    print(f"\n{'='*70}")
    print(f"Variable: {descripcion}")
    print(f"{'='*70}")
    
    # Tabla de contingencia
    tabla = pd.crosstab(df_filter[col], df_filter["label_gdm"])
    print("\nTabla de contingencia:")
    print(tabla)
    
    # Chi-cuadrado
    chi2, p, dof, exp = stats.chi2_contingency(tabla)
    
    # Calcular Odds Ratio (solo para tablas 2x2)
    OR = np.nan
    if tabla.shape == (2, 2):
        a, b, c, d = tabla.iloc[0,0], tabla.iloc[0,1], tabla.iloc[1,0], tabla.iloc[1,1]
        if (b * c) > 0:
            OR = (a * d) / (b * c)
    
    # Decisión
    decision = "Asociación significativa" if p < 0.05 else "Sin asociación"
    
    print(f"\nResultados:")
    print(f"  χ² = {chi2:.3f}")
    print(f"  gl = {dof}")
    print(f"  p = {p:.4f}")
    if not np.isnan(OR):
        print(f"  OR = {OR:.3f}")
    print(f"  Decisión: {decision}")
    
    # Guardar para tabla resumen
    chi_results.append({
        "Variable": descripcion,
        "Chi2": round(chi2, 3),
        "gl": dof,
        "p-value": round(p, 4),
        "Odds_Ratio": round(OR, 3) if not np.isnan(OR) else "N/A",
        "Decisión": decision
    })

# Tabla resumen
print(f"\n\n{'='*70}")
print("RESUMEN DE PRUEBAS CHI-CUADRADO")
print(f"{'='*70}\n")
df_chi_results = pd.DataFrame(chi_results)
display(df_chi_results)

# Guardar resultados
# df_chi_results.to_csv("chi_square_results.csv", index=False)

---

## 3. Sección 2.4 - Evaluación de Normalidad

### 3.1 Formato de presentación simplificado

**Ventajas del formato del compañero:**
- Bucle simple que evalúa múltiples variables
- Presentación clara con conclusión explícita ("PARECEN ser normales" vs "NO PARECEN")
- Fácil de leer en el output

**Tu implementación actual es más completa** (tienes Shapiro + KS), pero puedes mejorar la legibilidad:

In [ ]:
from scipy.stats import shapiro

variables_interes = [
    'bmi_prepreg_kg_m2',
    'fpg_mmol_l',
    'hba1c_percent',
    'systolic_bp_mmHg',
    'triglycerides_mmol_l',
    'hdl_mmol_l'
]

print("="*70)
print("EVALUACIÓN DE NORMALIDAD (Test de Shapiro-Wilk)")
print("="*70)

for col in variables_interes:
    # Limpiar datos
    datos_limpios = df_filter[col].dropna()
    
    # Test de Shapiro-Wilk
    stat, p_value = shapiro(datos_limpios)
    
    print(f"\nVariable: {col}")
    print(f"  N = {len(datos_limpios)}")
    print(f"  Estadístico W = {stat:.4f}")
    print(f"  P-value = {p_value:.4f}")
    
    if p_value > 0.05:
        print("  ✅ Conclusión: Los datos PARECEN ser normales (p > 0.05)")
    else:
        print("  ❌ Conclusión: Los datos NO PARECEN ser normales (p ≤ 0.05)")

---

## 4. Otras Utilidades

### 4.1 Análisis de outliers con IQR

**Su implementación es más simple que la tuya** (tú usas sistema de votación con 4 métodos, lo cual es SUPERIOR).  
Sin embargo, su código puede servir como alternativa simplificada si necesitas reportar outliers por variable individual:

In [ ]:
# Análisis de outliers variable por variable
numeric_cols = df_filter.select_dtypes(include=['float64', 'int64']).columns

for col_nombre in numeric_cols:
    print(f"------ {col_nombre} ------")
    col = df_filter[col_nombre].dropna()
    
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    
    if IQR == 0:
        print(f"⚠️  No se puede calcular outliers (IQR = 0)")
        continue
        
    limite_inferior = Q1 - (1.5 * IQR)
    limite_superior = Q3 + (1.5 * IQR)

    outliers = df_filter[(df_filter[col_nombre] < limite_inferior) | (df_filter[col_nombre] > limite_superior)]
    
    if outliers.empty:
        print("✅ No existen outliers")
    else:
        print(f"⚠️  {len(outliers)} outliers detectados")
        print(f"Porcentaje de outliers: {len(outliers) / len(df_filter) * 100:.2f}%")
        print(f"Rango válido: [{limite_inferior:.2f}, {limite_superior:.2f}]")
    
    print()

### 4.2 Visualizaciones compactas

**Su código de visualización es más compacto** que el tuyo (que es más detallado).  
Puede ser útil si necesitas generar gráficos rápidos para exploración:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ejemplo: Histogramas para todas las variables numéricas
numeric_cols = df_filter.select_dtypes(include=['float64', 'int64']).columns
n_cols = len(numeric_cols)

fig, axes = plt.subplots(1, n_cols, figsize=(5*n_cols, 4))

for idx, col in enumerate(numeric_cols):
    sns.histplot(data=df_filter, x=col, kde=True, ax=axes[idx])
    axes[idx].set_title(f'{col}')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

---

## 5. Resumen de Prioridades de Integración

### 🔴 CRÍTICO - Implementar inmediatamente en `Entregable.ipynb`:

1. **Chi-cuadrado para variables categóricas** (Sección 2.3)
   - `family_history_t2d`, `pcos`, `previous_gdm`, `smoking_first_trimester`
   - Código completo disponible en celda 2.3 arriba

2. **ANOVA/Kruskal-Wallis para dieta vs actividad física** (Sección 2.3)
   - Explícitamente pedido en instrucciones
   - Código completo disponible en celda 2.2 arriba

3. **Comparación Welch + Mann-Whitney para múltiples variables** (Sección 2.3)
   - Extender tu celda 52 actual
   - Código base disponible en celda 2.1 arriba

### 🟡 ALTA PRIORIDAD:

4. **Función `intervalo_confianza_95()`** (Sección 2.2)
   - Calcular IC para todas las variables continuas
   - Código disponible en celda 1.1 arriba

5. **Interpretaciones clínicas** (Sección 4.2)
   - Usar formato del compañero para escribir conclusiones
   - Ejemplos disponibles en celda 1.2 arriba

### 🟢 MEJORAS OPCIONALES:

6. **Formato de presentación de Shapiro-Wilk** (Sección 2.4)
   - Mejorar legibilidad de resultados
   - Código disponible en celda 3.1 arriba

---

## 6. Checklist de Implementación

**Antes de la entrega, asegurar:**

- [ ] Chi-cuadrado implementado y ejecutado para las 4 variables categóricas
- [ ] ANOVA/Kruskal-Wallis implementado para dieta vs actividad física
- [ ] Comparaciones paramétricas + no paramétricas para variables continuas clave
- [ ] IC calculados para todas las variables relevantes
- [ ] Interpretaciones clínicas escritas en Sección 4
- [ ] Tabla resumen de todas las pruebas de hipótesis
- [ ] Verificar que todas las celdas se ejecutan sin errores

---

**Fin del documento de referencia**